<a href="https://colab.research.google.com/github/Saumiyav01/huygvy/blob/main/BreastCancer_Survival_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_path = '/content/drive/MyDrive/BreastCancer_Survival/'

In [ ]:
!ls "/content/drive/MyDrive/BreastCancer_Survival/raw_data/"
!ls "/content/drive/MyDrive/BreastCancer_Survival/raw_data/clinical"
!ls "/content/drive/MyDrive/BreastCancer_Survival/raw_data/cnv"
!ls "/content/drive/MyDrive/BreastCancer_Survival/raw_data/fmpk"


clinical  cnv  fmpk
clinical.tsv  family_history.tsv  pathology_detail.tsv
exposure.tsv  follow_up.tsv
TCGA-BRCA.masked_cnv_DNAcopy.tsv
TCGA-BRCA.star_fpkm-uq.tsv


In [ ]:
import pandas as pd

# --- Clinical data ---
clin = pd.read_csv(base_path + 'clinical/clinical.tsv', sep='\t')
exposure = pd.read_csv(base_path + 'clinical/exposure.tsv', sep='\t')
family = pd.read_csv(base_path + 'clinical/family_history.tsv', sep='\t')
follow = pd.read_csv(base_path + 'clinical/follow_up.tsv', sep='\t')
pathology = pd.read_csv(base_path + 'clinical/pathology_detail.tsv', sep='\t')

# --- CNV & RNA files ---
cnv = pd.read_csv(base_path + 'cnv/TCGA-BRCA.masked_cnv_DNAcopy.tsv', sep='\t')
rna = pd.read_csv(base_path + 'fmpk/TCGA-BRCA.star_fpkm-uq.tsv', sep='\t')

print("✅ All files loaded successfully!")
print("Clinical:", clin.shape)
print("Exposure:", exposure.shape)
print("CNV:", cnv.shape)
print("RNA:", rna.shape)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/BreastCancer_Survival/clinical/clinical.tsv'

In [ ]:
!ls "/content/drive/MyDrive/BreastCancer_Survival"


raw_data


In [ ]:


import pandas as pd

# --- Clinical data ---
clin = pd.read_csv(base_path + 'clinical/clinical.tsv', sep='\t')
exposure = pd.read_csv(base_path + 'clinical/exposure.tsv', sep='\t')
family = pd.read_csv(base_path + 'clinical/family_history.tsv', sep='\t')
follow = pd.read_csv(base_path + 'clinical/follow_up.tsv', sep='\t')
pathology = pd.read_csv(base_path + 'clinical/pathology_detail.tsv', sep='\t')

# --- CNV & RNA files ---
cnv = pd.read_csv(base_path + 'cnv/TCGA-BRCA.masked_cnv_DNAcopy.tsv', sep='\t')
rna = pd.read_csv(base_path + 'fmpk/TCGA-BRCA.star_fpkm-uq.tsv', sep='\t')

print("All files loaded successfully!")
print("Clinical:", clin.shape)
print("Exposure:", exposure.shape)
print("CNV:", cnv.shape)
print("RNA:", rna.shape)


/tmp/ipython-input-3312245549.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  clin = pd.read_csv(base_path + 'clinical/clinical.tsv', sep='\t')


✅ All files loaded successfully!
Clinical: (5546, 210)
Exposure: (1, 40)
CNV: (381953, 5)
RNA: (60660, 1227)


In [ ]:

import pandas as pd

clin = pd.read_csv(base_path + 'clinical/clinical.tsv', sep='\t', low_memory=False)
exposure = pd.read_csv(base_path + 'clinical/exposure.tsv', sep='\t')
family = pd.read_csv(base_path + 'clinical/family_history.tsv', sep='\t')
follow = pd.read_csv(base_path + 'clinical/follow_up.tsv', sep='\t')
pathology = pd.read_csv(base_path + 'clinical/pathology_detail.tsv', sep='\t')


for df in [clin, exposure, family, follow, pathology]:
    if 'case_submitter_id' in df.columns:
        df['case_submitter_id'] = df['case_submitter_id'].astype(str).str[:12]


clinical_merged = clin.copy()
for sub_df in [exposure, family, follow, pathology]:
    if 'case_submitter_id' in sub_df.columns:
        clinical_merged = clinical_merged.merge(sub_df, on='case_submitter_id', how='left')

clinical_merged.to_csv('/content/drive/MyDrive/BreastCancer_Survival/clinical_merged.csv', index=False)
print("✅ Clinical files merged successfully! Shape:", clinical_merged.shape)


✅ Clinical files merged successfully! Shape: (5546, 210)


In [ ]:

def clean_tcga_id(x):
    return str(x)[:12]


rna['sample_id'] = rna['sample_id'].apply(clean_tcga_id)
cnv['sample_id'] = cnv['sample_id'].apply(clean_tcga_id)

print("✅ Sample IDs standardized in RNA and CNV datasets.")


KeyError: 'sample_id'

In [ ]:
print("RNA columns:\n", rna.columns.tolist()[:20])
print("\nCNV columns:\n", cnv.columns.tolist()[:20])


RNA columns:
 ['Ensembl_ID', 'TCGA-D8-A146-01A', 'TCGA-AQ-A0Y5-01A', 'TCGA-C8-A274-01A', 'TCGA-BH-A0BD-01A', 'TCGA-B6-A1KC-01B', 'TCGA-AC-A62V-01A', 'TCGA-AO-A0J5-01A', 'TCGA-BH-A0B1-01A', 'TCGA-A2-A0YM-01A', 'TCGA-AO-A03N-01B', 'TCGA-AO-A1KQ-01A', 'TCGA-E2-A1LI-01A', 'TCGA-BH-A18L-11A', 'TCGA-B6-A0WV-01A', 'TCGA-E2-A1LE-01A', 'TCGA-A2-A0CO-01A', 'TCGA-AN-A0AK-01A', 'TCGA-BH-A28Q-01A', 'TCGA-3C-AALJ-01A']

CNV columns:
 ['sample', 'Chrom', 'Start', 'End', 'value']


In [ ]:

rna = rna.set_index('Ensembl_ID')

rna = rna.T.reset_index()


rna.rename(columns={'index': 'sample_id'}, inplace=True)

print("✅ RNA data reshaped successfully!")
print("RNA shape after transpose:", rna.shape)
print("RNA columns preview:", rna.columns[:5])


cnv.rename(columns={'sample': 'sample_id'}, inplace=True)

print("✅ CNV column renamed successfully!")
print("CNV columns now:", cnv.columns.tolist())


✅ RNA data reshaped successfully!
RNA shape after transpose: (1226, 60661)
RNA columns preview: Index(['sample_id', 'ENSG00000000003.15', 'ENSG00000000005.6',
       'ENSG00000000419.13', 'ENSG00000000457.14'],
      dtype='object', name='Ensembl_ID')
✅ CNV column renamed successfully!
CNV columns now: ['sample_id', 'Chrom', 'Start', 'End', 'value']


In [ ]:
common_ids = set(rna['sample_id']) & set(cnv['sample_id']) & set(clinical_merged['case_submitter_id'])
print("✅ Common patients found across all datasets:", len(common_ids))

rna = rna[rna['sample_id'].isin(common_ids)]
cnv = cnv[cnv['sample_id'].isin(common_ids)]
clinical_merged = clinical_merged[clinical_merged['case_submitter_id'].isin(common_ids)]


KeyError: 'case_submitter_id'

In [ ]:
print("Clinical columns:", clin.columns.tolist()[:30])


Clinical columns: ['project.project_id', 'cases.case_id', 'cases.consent_type', 'cases.days_to_consent', 'cases.days_to_lost_to_followup', 'cases.disease_type', 'cases.index_date', 'cases.lost_to_followup', 'cases.primary_site', 'cases.submitter_id', 'demographic.age_at_index', 'demographic.age_is_obfuscated', 'demographic.cause_of_death', 'demographic.cause_of_death_source', 'demographic.country_of_birth', 'demographic.country_of_residence_at_enrollment', 'demographic.days_to_birth', 'demographic.days_to_death', 'demographic.demographic_id', 'demographic.education_level', 'demographic.ethnicity', 'demographic.gender', 'demographic.marital_status', 'demographic.occupation_duration_years', 'demographic.population_group', 'demographic.premature_at_birth', 'demographic.race', 'demographic.submitter_id', 'demographic.vital_status', 'demographic.weeks_gestation_at_birth']


In [ ]:
# ✅ Standardize ID column names across all clinical tables
for df in [clin, exposure, family, follow, pathology]:
    if 'cases.submitter_id' in df.columns:
        df.rename(columns={'cases.submitter_id': 'case_submitter_id'}, inplace=True)
    if 'demographic.submitter_id' in df.columns:
        # This field is usually the same patient ID, but we’ll prioritize 'cases.submitter_id'
        df.rename(columns={'demographic.submitter_id': 'case_submitter_id'}, inplace=True)
    if 'bcr_patient_barcode' in df.columns:
        df.rename(columns={'bcr_patient_barcode': 'case_submitter_id'}, inplace=True)
    if 'submitter_id' in df.columns and 'case_submitter_id' not in df.columns:
        df.rename(columns={'submitter_id': 'case_submitter_id'}, inplace=True)
print("✅ ID column check in main clinical file:")
print(clin.columns[:10])


✅ ID column check in main clinical file:
Index(['project.project_id', 'cases.case_id', 'cases.consent_type',
       'cases.days_to_consent', 'cases.days_to_lost_to_followup',
       'cases.disease_type', 'cases.index_date', 'cases.lost_to_followup',
       'cases.primary_site', 'case_submitter_id'],
      dtype='object')


In [ ]:
# Check how many unique IDs exist in each dataset
print(" RNA unique IDs:", rna['sample_id'].nunique())
print(" CNV unique IDs:", cnv['sample_id'].nunique())
print(" Clinical unique IDs:", clin['case_submitter_id'].nunique())

# Check ID overlap
common_ids = set(rna['sample_id']) & set(cnv['sample_id']) & set(clin['case_submitter_id'])
print("✅ Common patients across all datasets:", len(common_ids))


 RNA unique IDs: 1226
 CNV unique IDs: 1111
 Clinical unique IDs: case_submitter_id    1098
case_submitter_id    1098
dtype: int64
✅ Common patients across all datasets: 0


In [ ]:
# --- Function to keep only first 12 chars (patient-level ID) ---
def clean_tcga_id(x):
    return str(x)[:12]

# Apply to all relevant datasets
rna['sample_id'] = rna['sample_id'].apply(clean_tcga_id)
cnv['sample_id'] = cnv['sample_id'].apply(clean_tcga_id)
clin['case_submitter_id'] = clin['case_submitter_id'].apply(clean_tcga_id)


/tmp/ipython-input-4283386944.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  clin['case_submitter_id'] = clin['case_submitter_id'].apply(clean_tcga_id)


In [ ]:
print(" RNA unique IDs:", rna['sample_id'].nunique())
print("CNV unique IDs:", cnv['sample_id'].nunique())
print(" Clinical unique IDs:", clin['case_submitter_id'].nunique())

common_ids = set(rna['sample_id']) & set(cnv['sample_id']) & set(clin['case_submitter_id'])
print("✅ Common patients across all datasets:", len(common_ids))


 RNA unique IDs: 1095
CNV unique IDs: 1096
 Clinical unique IDs: case_submitter_id    1
case_submitter_id    1
dtype: int64
✅ Common patients across all datasets: 0


In [ ]:
print(clin['case_submitter_id'].head(20))


   case_submitter_id case_submitter_id
0       0       TCGA      0       TCGA
1       0       TCGA      0       TCGA
2       0       TCGA      0       TCGA
3       0       TCGA      0       TCGA
4       0       TCGA      0       TCGA
5       0       TCGA      0       TCGA
6       0       TCGA      0       TCGA
7       0       TCGA      0       TCGA
8       0       TCGA      0       TCGA
9       0       TCGA      0       TCGA
10      0       TCGA      0       TCGA
11      0       TCGA      0       TCGA
12      0       TCGA      0       TCGA
13      0       TCGA      0       TCGA
14      0       TCGA      0       TCGA
15      0       TCGA      0       TCGA
16      0       TCGA      0       TCGA
17      0       TCGA      0       TCGA
18      0       TCGA      0       TCGA
19      0       TCGA      0       TCGA


In [ ]:
[id for id in clin.columns if 'submitter_id' in id]



['case_submitter_id',
 'case_submitter_id',
 'diagnoses.submitter_id',
 'treatments.submitter_id']

In [ ]:
for col in ['diagnoses.submitter_id', 'treatments.submitter_id']:
    if col in clin.columns:
        print(f"\n {col} → sample values:")
        print(clin[col].dropna().head(10).tolist())



 diagnoses.submitter_id → sample values:
['TCGA-E2-A1IU_diagnosis', 'TCGA-E2-A1IU_diagnosis', 'TCGA-E2-A1IU_diagnosis', 'TCGA-A1-A0SB_diagnosis', 'TCGA-A2-A04W_diagnosis', 'TCGA-A2-A04W_diagnosis', 'TCGA-A2-A04W_diagnosis', 'TCGA-A2-A04W_diagnosis', 'TCGA-A2-A04W_diagnosis', 'TCGA-A2-A04W_diagnosis']

 treatments.submitter_id → sample values:
['TCGA-E2-A1IU_treatment2', 'TCGA-E2-A1IU_treatment', 'TCGA-E2-A1IU_treatment3', 'TCGA-A1-A0SB_treatment', 'TCGA-A2-A04W_treatment5', 'TCGA-A2-A04W_treatment3', 'TCGA-A2-A04W_treatment', 'TCGA-A2-A04W_treatment7', 'TCGA-A2-A04W_treatment6', 'TCGA-A2-A04W_treatment4']


In [ ]:
import re

# Prefer diagnoses.submitter_id if available, otherwise use treatments.submitter_id
if 'diagnoses.submitter_id' in clin.columns:
    clin['case_submitter_id'] = clin['diagnoses.submitter_id']
elif 'treatments.submitter_id' in clin.columns:
    clin['case_submitter_id'] = clin['treatments.submitter_id']

# Clean TCGA IDs: remove "_diagnosis", "_treatment", and any suffixes
def extract_tcga_id(x):
    if pd.isna(x):
        return None
    match = re.match(r'(TCGA-\w\w-\w\w\w\w)', str(x))
    return match.group(1) if match else None

clin['case_submitter_id'] = clin['case_submitter_id'].apply(extract_tcga_id)

print("✅ Extracted clean TCGA case IDs:")
print(clin['case_submitter_id'].dropna().head(10))
print("Unique patient IDs:", clin['case_submitter_id'].nunique())


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
import re
import pandas as pd

# --- Step 1: choose which column to use (diagnoses or treatments) ---
if 'diagnoses.submitter_id' in clin.columns:
    source_col = 'diagnoses.submitter_id'
elif 'treatments.submitter_id' in clin.columns:
    source_col = 'treatments.submitter_id'
else:
    raise ValueError("No valid submitter_id column found in clinical data")

print(f"✅ Using column: {source_col} for patient IDs")

# --- Step 2: extract clean TCGA IDs from it ---
def extract_tcga_id(x):
    if pd.isna(x):
        return None
    match = re.match(r'(TCGA-\w\w-\w\w\w\w)', str(x))
    return match.group(1) if match else None

clin['case_submitter_id'] = clin[source_col].apply(extract_tcga_id)

# --- Step 3: sanity check ---
print("✅ Extracted clean TCGA case IDs:")
print(clin['case_submitter_id'].dropna().head(10))
print("Unique patient IDs:", clin['case_submitter_id'].nunique())


✅ Using column: diagnoses.submitter_id for patient IDs
✅ Extracted clean TCGA case IDs:
  case_submitter_id case_submitter_id
0      TCGA-E2-A1IU      TCGA-E2-A1IU
1      TCGA-E2-A1IU      TCGA-E2-A1IU
2      TCGA-E2-A1IU      TCGA-E2-A1IU
3      TCGA-A1-A0SB      TCGA-A1-A0SB
4      TCGA-A2-A04W      TCGA-A2-A04W
5      TCGA-A2-A04W      TCGA-A2-A04W
6      TCGA-A2-A04W      TCGA-A2-A04W
7      TCGA-A2-A04W      TCGA-A2-A04W
8      TCGA-A2-A04W      TCGA-A2-A04W
9      TCGA-A2-A04W      TCGA-A2-A04W
Unique patient IDs: case_submitter_id    1097
case_submitter_id    1097
dtype: int64


In [ ]:
common_ids = set(rna['sample_id']) & set(cnv['sample_id']) & set(clin['case_submitter_id'])
print("Common patients across all datasets:", len(common_ids))


Common patients across all datasets: 0


In [ ]:
print("RNA IDs:", list(rna['sample_id'].dropna().unique())[:10])
print("CNV IDs:", list(cnv['sample_id'].dropna().unique())[:10])
print("Clinical IDs:", list(clin['case_submitter_id'].dropna().unique())[:10])


RNA IDs: ['TCGA-D8-A146', 'TCGA-AQ-A0Y5', 'TCGA-C8-A274', 'TCGA-BH-A0BD', 'TCGA-B6-A1KC', 'TCGA-AC-A62V', 'TCGA-AO-A0J5', 'TCGA-BH-A0B1', 'TCGA-A2-A0YM', 'TCGA-AO-A03N']
CNV IDs: ['TCGA-C8-A1HO', 'TCGA-A7-A0CD', 'TCGA-A8-A07E', 'TCGA-AC-A2QH', 'TCGA-E2-A1L7', 'TCGA-AO-A1KQ', 'TCGA-D8-A143', 'TCGA-LL-A442', 'TCGA-E2-A1LI', 'TCGA-D8-A27R']


AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
print(clin.columns.tolist())


['project.project_id', 'cases.case_id', 'cases.consent_type', 'cases.days_to_consent', 'cases.days_to_lost_to_followup', 'cases.disease_type', 'cases.index_date', 'cases.lost_to_followup', 'cases.primary_site', 'case_submitter_id', 'demographic.age_at_index', 'demographic.age_is_obfuscated', 'demographic.cause_of_death', 'demographic.cause_of_death_source', 'demographic.country_of_birth', 'demographic.country_of_residence_at_enrollment', 'demographic.days_to_birth', 'demographic.days_to_death', 'demographic.demographic_id', 'demographic.education_level', 'demographic.ethnicity', 'demographic.gender', 'demographic.marital_status', 'demographic.occupation_duration_years', 'demographic.population_group', 'demographic.premature_at_birth', 'demographic.race', 'case_submitter_id', 'demographic.vital_status', 'demographic.weeks_gestation_at_birth', 'demographic.year_of_birth', 'demographic.year_of_death', 'diagnoses.adrenal_hormone', 'diagnoses.age_at_diagnosis', 'diagnoses.ajcc_clinical_m', 

In [ ]:
import re

# --- Step 1: extract the true TCGA case ID ---
def extract_tcga_case(x):
    if pd.isna(x):
        return None
    x = str(x).strip().upper().replace('_', '-')
    match = re.search(r'TCGA-[A-Z0-9]{2}-[A-Z0-9]{4}', x)
    return match.group(0) if match else None

# --- Step 2: apply to diagnoses.submitter_id ---
clin['case_submitter_id'] = clin['diagnoses.submitter_id'].apply(extract_tcga_case)

# --- Step 3: sanity check ---
print("✅ Extracted clean TCGA IDs from diagnoses.submitter_id:")
print(clin['case_submitter_id'].dropna().head(10))
print("Unique patient IDs in clinical:", clin['case_submitter_id'].nunique())


✅ Extracted clean TCGA IDs from diagnoses.submitter_id:
  case_submitter_id case_submitter_id
0      TCGA-E2-A1IU      TCGA-E2-A1IU
1      TCGA-E2-A1IU      TCGA-E2-A1IU
2      TCGA-E2-A1IU      TCGA-E2-A1IU
3      TCGA-A1-A0SB      TCGA-A1-A0SB
4      TCGA-A2-A04W      TCGA-A2-A04W
5      TCGA-A2-A04W      TCGA-A2-A04W
6      TCGA-A2-A04W      TCGA-A2-A04W
7      TCGA-A2-A04W      TCGA-A2-A04W
8      TCGA-A2-A04W      TCGA-A2-A04W
9      TCGA-A2-A04W      TCGA-A2-A04W
Unique patient IDs in clinical: case_submitter_id    1097
case_submitter_id    1097
dtype: int64


In [ ]:
common_ids = set(rna['sample_id']) & set(cnv['sample_id']) & set(clin['case_submitter_id'])
print("✅ Common patients across all datasets:", len(common_ids))


✅ Common patients across all datasets: 0


In [ ]:
print("RNA sample IDs (first 10):")
print(rna['sample_id'].dropna().unique()[:10])

print("\nCNV sample IDs (first 10):")
print(cnv['sample_id'].dropna().unique()[:10])

print("\nClinical case IDs (first 10):")
print(clin['case_submitter_id'].dropna().unique()[:10])


RNA sample IDs (first 10):
['TCGA-D8-A146' 'TCGA-AQ-A0Y5' 'TCGA-C8-A274' 'TCGA-BH-A0BD'
 'TCGA-B6-A1KC' 'TCGA-AC-A62V' 'TCGA-AO-A0J5' 'TCGA-BH-A0B1'
 'TCGA-A2-A0YM' 'TCGA-AO-A03N']

CNV sample IDs (first 10):
['TCGA-C8-A1HO' 'TCGA-A7-A0CD' 'TCGA-A8-A07E' 'TCGA-AC-A2QH'
 'TCGA-E2-A1L7' 'TCGA-AO-A1KQ' 'TCGA-D8-A143' 'TCGA-LL-A442'
 'TCGA-E2-A1LI' 'TCGA-D8-A27R']

Clinical case IDs (first 10):


AttributeError: 'DataFrame' object has no attribute 'unique'

In [ ]:
[col for col in clin.columns if col == "case_submitter_id"]


['case_submitter_id', 'case_submitter_id']

In [ ]:
# Find all duplicate 'case_submitter_id' columns
dup_cols = [c for c in clin.columns if c == 'case_submitter_id']

# If there are duplicates, keep only the last one (the correct cleaned version)
if len(dup_cols) > 1:
    # Keep the last occurrence
    clin = clin.loc[:, ~clin.columns.duplicated(keep='last')]
    print("✅ Dropped duplicate 'case_submitter_id' columns. Remaining:", [c for c in clin.columns if 'submitter_id' in c])
else:
    print("No duplicates found — safe to proceed.")


✅ Dropped duplicate 'case_submitter_id' columns. Remaining: ['case_submitter_id', 'diagnoses.submitter_id', 'treatments.submitter_id']


In [ ]:
print(clin['case_submitter_id'].dropna().unique()[:10])
print("Unique Clinical IDs:", clin['case_submitter_id'].nunique())


['TCGA-E2-A1IU' 'TCGA-A1-A0SB' 'TCGA-A2-A04W' 'TCGA-AN-A0AM'
 'TCGA-LL-A440' 'TCGA-A7-A26E' 'TCGA-A8-A07W' 'TCGA-D8-A1XY'
 'TCGA-EW-A1P7' 'TCGA-AO-A128']
Unique Clinical IDs: 1097


In [ ]:
common_ids = set(rna['sample_id']) & set(cnv['sample_id']) & set(clin['case_submitter_id'])
print("✅ Common patients across all datasets:", len(common_ids))


✅ Common patients across all datasets: 1092


In [ ]:
# Keep only shared patient IDs
rna = rna[rna['sample_id'].isin(common_ids)]
cnv = cnv[cnv['sample_id'].isin(common_ids)]
clinical_merged = clin[clin['case_submitter_id'].isin(common_ids)]

print("RNA shape:", rna.shape)
print("CNV shape:", cnv.shape)
print("Clinical shape:", clinical_merged.shape)


RNA shape: (1222, 60661)
CNV shape: (380616, 5)
Clinical shape: (5519, 209)


In [ ]:
rna = rna[rna['sample_id'].isin(common_ids)]
print("RNA filtered shape:", rna.shape)


RNA filtered shape: (1222, 60661)


In [ ]:
cnv_agg = cnv.groupby('sample_id')['value'].mean().reset_index()
print("CNV aggregated shape:", cnv_agg.shape)


CNV aggregated shape: (1092, 2)


In [ ]:
clinical_unique = clinical_merged.drop_duplicates(subset=['case_submitter_id'], keep='first')
print("Clinical unique shape:", clinical_unique.shape)


Clinical unique shape: (1092, 209)


In [ ]:
# Merge RNA + CNV
combined = rna.merge(cnv_agg, on='sample_id', how='inner', suffixes=('_rna', '_cnv'))

# Merge Clinical
combined = combined.merge(
    clinical_unique,
    left_on='sample_id',
    right_on='case_submitter_id',
    how='inner'
)

print("✅ Combined dataset created successfully! Shape:", combined.shape)


✅ Combined dataset created successfully! Shape: (1222, 60871)


In [ ]:
# --- Survival label creation ---
import numpy as np

# Event column: 1 = Dead, 0 = Alive
combined['event'] = combined['demographic.vital_status'].map({'Dead': 1, 'Alive': 0})

# Time to event (in days)
combined['time'] = combined[['demographic.days_to_death', 'diagnoses.days_to_last_follow_up']].apply(
    lambda x: x['demographic.days_to_death'] if not pd.isna(x['demographic.days_to_death']) else x['diagnoses.days_to_last_follow_up'],
    axis=1
)

# Drop rows with missing survival information
combined = combined.dropna(subset=['event', 'time'])
combined['time'] = combined['time'].astype(float)

# Keep only valid positive survival times
combined = combined[combined['time'] > 0]

print("✅ Survival columns created successfully!")
print("Number of patients with survival info:", combined.shape[0])
print(combined[['sample_id', 'event', 'time']].head(10))


ValueError: could not convert string to float: "'--"

In [ ]:
import numpy as np
import pandas as pd

# --- Replace non-numeric placeholders like '--', 'NA', etc. ---
combined['demographic.days_to_death'] = combined['demographic.days_to_death'].replace(['--', 'NA', 'N/A', 'na', ''], np.nan)
combined['diagnoses.days_to_last_follow_up'] = combined['diagnoses.days_to_last_follow_up'].replace(['--', 'NA', 'N/A', 'na', ''], np.nan)

# Convert to numeric safely
combined['demographic.days_to_death'] = pd.to_numeric(combined['demographic.days_to_death'], errors='coerce')
combined['diagnoses.days_to_last_follow_up'] = pd.to_numeric(combined['diagnoses.days_to_last_follow_up'], errors='coerce')

# Event: 1 = Dead, 0 = Alive
combined['event'] = combined['demographic.vital_status'].map({'Dead': 1, 'Alive': 0})

# Time to event (take whichever is available)
combined['time'] = combined.apply(
    lambda x: x['demographic.days_to_death']
    if not pd.isna(x['demographic.days_to_death'])
    else x['diagnoses.days_to_last_follow_up'],
    axis=1
)

# Remove missing or invalid values
combined = combined.dropna(subset=['time', 'event'])
combined['time'] = combined['time'].astype(float)
combined = combined[combined['time'] > 0]

print("✅ Clean survival times created successfully!")
print("Number of patients with survival info:", combined.shape[0])
print(combined[['sample_id', 'event', 'time']].head(10))


✅ Clean survival times created successfully!
Number of patients with survival info: 1132
       sample_id  event    time
0   TCGA-D8-A146      0   643.0
1   TCGA-AQ-A0Y5      1   172.0
2   TCGA-C8-A274      0   508.0
3   TCGA-BH-A0BD      0   554.0
4   TCGA-B6-A1KC      0  1326.0
5   TCGA-AC-A62V      1   348.0
6   TCGA-AO-A0J5      1   792.0
7   TCGA-BH-A0B1      0  1148.0
8   TCGA-A2-A0YM      0   965.0
10  TCGA-AO-A1KQ      0  1882.0


In [ ]:
final_path = '/content/drive/MyDrive/BreastCancer_Survival/final_ready_dataset.csv'
combined.to_csv(final_path, index=False)
print("💾 Final modeling dataset saved to:", final_path)
print("Final dataset shape:", combined.shape)


💾 Final modeling dataset saved to: /content/drive/MyDrive/BreastCancer_Survival/final_ready_dataset.csv
Final dataset shape: (1132, 60873)


In [ ]:
# Double-check if 'combined' dataframe exists
print("combined shape:", combined.shape)
combined.head()

# Ensure directory exists
import os
os.makedirs('/content/drive/MyDrive/BreastCancer_Survival/processed_data', exist_ok=True)

# Save the preprocessed dataset
combined.to_csv('/content/drive/MyDrive/BreastCancer_Survival/processed_data/combined_dataset.csv', index=False)
print("✅ Saved successfully!")


combined shape: (1132, 60873)
✅ Saved successfully!


In [ ]:
!ls '/content/drive/MyDrive/BreastCancer_Survival/processed_data'


clinical_merged.csv  combined_dataset.csv
